<a href="https://colab.research.google.com/github/luc-sol-2048/dw_matrix_road_sign/blob/master/day_5_m3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
#!pip install hyperopt

ERROR! Session/line number was not unique in database. History logging moved to new session 59


In [34]:
import pandas as pd
import numpy as np
import tensorflow as tf
import os
import datetime
import matplotlib.pyplot as plt


from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D,MaxPool2D,Dense,Flatten,Dropout
from tensorflow.keras.utils import to_categorical
from skimage import color,exposure
from sklearn.metrics import accuracy_score
from hyperopt import hp, STATUS_OK, tpe, Trials, fmin
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [35]:
cd '/content/drive/My Drive/Colab Notebooks/dw_matrix/matrix_three/dw_matrix_road_sign/'

/content/drive/My Drive/Colab Notebooks/dw_matrix/matrix_three/dw_matrix_road_sign


In [0]:
train = pd.read_pickle('./data/train.p')
X_train, y_train = train['features'], train['labels']

test = pd.read_pickle('./data/test.p')
X_test, y_test = test['features'], test['labels']

df = pd.read_csv('./data/signnames.csv')
sign_labels = df.to_dict()['b']


In [0]:
if y_train.ndim ==1 : y_train = to_categorical(y_train)
if y_test.ndim ==1 : y_test = to_categorical(y_test)

input_shape = X_train.shape[1:]
num_classes = y_train.shape[1]

In [0]:
def train_model(model, X_train, y_train, params_fit={}):
  model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics = ['accuracy'])
  
  
  model.fit(
      X_train,
      y_train,
      batch = params_fit.get('batch_size',128),
      epochs = params_fit.get('epochs',5),


      
  )
  
  return model

In [0]:
def predict_model(model_trained, X_test,y_test,scoring=accuracy_score):
  y_test_norm = np.argmax(y_test, axis=1)

  y_pred_prob = model_trained.predict(X_test)
  y_pred = np.argmax(y_pred_prob, axis=1)

  return scoring(y_test_norm,y_pred)

In [0]:
def get_cnn_v5(input_shape, num_classes):
  return Sequential([
      Conv2D(filters=32,kernel_size=(3,3),activation='relu',input_shape = input_shape),
      Conv2D(filters=32,kernel_size=(3,3),activation='relu',padding='same'),
      MaxPool2D(),
      Dropout(0.3),
      
      Conv2D(filters=64,kernel_size=(3,3),activation='relu',padding='same'),
      Conv2D(filters=64,kernel_size=(3,3),activation='relu'),
      MaxPool2D(),
      Dropout(0.3),
      
      Conv2D(filters=64,kernel_size=(3,3),activation='relu',padding='same'),
      Conv2D(filters=64,kernel_size=(3,3),activation='relu'),
      MaxPool2D(),
      Dropout(0.3),
            
      Flatten(),

      Dense(1024,activation='relu'),
      Dropout(0.3),

      Dense(1024,activation='relu'),
      Dropout(0.3),

      Dense(num_classes, activation='softmax'),
  ]
  )

In [45]:
model = get_cnn_v5(input_shape,num_classes)
model_trained = train_model(model,X_train,y_train)

predict_model(model_trained, X_test, y_test)

Epoch 1/5
1088/1088 [==============================] - 5s 5ms/step - loss: 1.7900 - accuracy: 0.4709
Epoch 2/5
1088/1088 [==============================] - 5s 5ms/step - loss: 0.5469 - accuracy: 0.8300
Epoch 3/5
1088/1088 [==============================] - 5s 5ms/step - loss: 0.3720 - accuracy: 0.8910
Epoch 4/5
1088/1088 [==============================] - 5s 5ms/step - loss: 0.3198 - accuracy: 0.9111
Epoch 5/5
1088/1088 [==============================] - 5s 5ms/step - loss: 0.2897 - accuracy: 0.9215


0.963265306122449

In [0]:
def get_cnn_v5(params):
  return Sequential([
      Conv2D(filters=32,kernel_size=(3,3),activation='relu',input_shape = input_shape),
      Conv2D(filters=32,kernel_size=(3,3),activation='relu',padding='same'),
      MaxPool2D(),
      Dropout(params['dropout_cnn_block_one']),
      
      Conv2D(filters=64,kernel_size=(3,3),activation='relu',padding='same'),
      Conv2D(filters=64,kernel_size=(3,3),activation='relu'),
      MaxPool2D(),
      Dropout(params['dropout_cnn_block_two']),
      
      Conv2D(filters=128,kernel_size=(3,3),activation='relu',padding='same'),
      Conv2D(filters=128,kernel_size=(3,3),activation='relu'),
      MaxPool2D(),
      Dropout(params['dropout_cnn_block_three']),
            
      Flatten(),

      Dense(1024,activation='relu'),
      Dropout(params['dropout_dense_block_one']),

      Dense(1024,activation='relu'),
      Dropout(params['dropout_dense_block_two']),

      Dense(num_classes, activation='softmax'),
  ]
  )

In [0]:
def func_obj(params):
  model = get_cnn_v5(params)
  model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics = ['accuracy'])
  
  model.fit(
      X_train,
      y_train,
      batch_size = int(params.get('batch_size',128)),
      epochs = 5,
      verbose = 0
      )
  
  score = model.evaluate(X_test, y_test,verbose=0)
  accuracy = score[1]

  print(params, 'accuracy={}'.format(accuracy))

  return {'loss': -accuracy, 'status': STATUS_OK, 'model':model}

In [49]:
space = {
    'batch_size': hp.quniform('batch_size',100,200,10),
    'dropout_cnn_block_one': hp.uniform('dropout_cnn_block_one', 0.3, 0.5),
    'dropout_cnn_block_two': hp.uniform('dropout_cnn_block_two', 0.3, 0.5),
    'dropout_cnn_block_three': hp.uniform('dropout_cnn_block_three', 0.3, 0.5,),

    'dropout_dense_block_one': hp.uniform('dropout_dense_block_one', 0.3, 0.7),
    'dropout_dense_block_two': hp.uniform('dropout_dense_block_two', 0.3, 0.7)
}

best = fmin(
    func_obj,
    space,
    tpe.suggest,
    30,
    Trials()
)

{'batch_size': 200.0, 'dropout_cnn_block_one': 0.3086331853769426, 'dropout_cnn_block_three': 0.4027133005041521, 'dropout_cnn_block_two': 0.4494376505867658, 'dropout_dense_block_one': 0.5282753931380186, 'dropout_dense_block_two': 0.3261657425590439}
accuracy=0.9131519198417664
{'batch_size': 170.0, 'dropout_cnn_block_one': 0.4970499404284838, 'dropout_cnn_block_three': 0.3952469147534923, 'dropout_cnn_block_two': 0.3080562157553827, 'dropout_dense_block_one': 0.6110673482437669, 'dropout_dense_block_two': 0.4028813988740473}
accuracy=0.7891156673431396
{'batch_size': 120.0, 'dropout_cnn_block_one': 0.37639255200124533, 'dropout_cnn_block_three': 0.3312712431292111, 'dropout_cnn_block_two': 0.33883210581440665, 'dropout_dense_block_one': 0.45595938244184925, 'dropout_dense_block_two': 0.47450997697005454}
accuracy=0.9358276724815369
{'batch_size': 190.0, 'dropout_cnn_block_one': 0.3968565629342464, 'dropout_cnn_block_three': 0.4943212646191046, 'dropout_cnn_block_two': 0.490431921608